In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging

module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pathlib import Path

import torch
import torch.utils.data
from torchvision import transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
"""matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})"""

import pvi
from pvi.models import ClassificationBNNLocalRepam
from pvi.clients import Client
from pvi.servers import SequentialServer, SynchronousServer
from pvi.distributions import MeanFieldGaussianDistribution, MeanFieldGaussianFactor
from pvi.utils.training_utils import EarlyStopping

In [2]:
data_dir = Path("../../../data")
cache_dir = Path("../../../experiment_cache/femnist")

## Define various functions for splitting data and recording performance metrics.

In [3]:
# Data splitting functions.
def homogeneous_split(data, num_clients=100, seed=42):
    # Set numpy's random seed.
    np.random.seed(seed)
    
    perm = np.random.permutation(len(data["x"]))
    client_data = []
    for i in range(num_clients):
        client_idx = perm[i::num_clients]
        client_data.append({"x": data["x"][client_idx], "y": data["y"][client_idx]})
    
    return client_data


# Performance metric function. Why is the batch size 512 when the split gives 600? => Mini batches?
def performance_metrics(client, data, batch_size=512):
    dataset = torch.utils.data.TensorDataset(data["x"], data["y"])
    loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
    
    device = client.config["device"]
    
    if device == "cuda":
        loader.pin_memory = True
        
    preds, mlls = [], []
    for (x_batch, y_batch) in loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        pp = client.model_predict(x_batch)
        preds.append(pp.component_distribution.probs.mean(1).cpu())
        mlls.append(pp.log_prob(y_batch).cpu())
        
    mll = torch.cat(mlls).mean()
    preds = torch.cat(preds)
    acc = sum(torch.argmax(preds, dim=-1) == loader.dataset.tensors[1]) / len(
        loader.dataset.tensors[1]
    )
    
    return {"mll": mll, "acc": acc}

## Data preprocessing

In [4]:
transform_train = transforms.Compose([transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_set = datasets.MNIST(root=data_dir, train=True, download=True, transform=transform_train)
test_set = datasets.MNIST(root=data_dir, train=False, download=True, transform=transform_test)

train_data = {
    "x": ((train_set.data - 0) / 255).reshape(-1, 28 * 28),
    "y": train_set.targets,
}

test_data = {
    "x": ((test_set.data - 0) / 255).reshape(-1, 28 * 28),
    "y": test_set.targets,
}

In [5]:
train_data["x"]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [6]:
# Get client splits.
client_data = homogeneous_split(train_data, 10, seed=42)


In [7]:
len(client_data[0]["x"])

6000

## Define configuration for server and clients
## $\eta=0.002$

In [8]:
model_config = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 2e-3},
    "sigma_optimiser_params": {"lr": 2e-3},
    "early_stopping": EarlyStopping(5, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 2500,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "AR",
    "alpha": 0.5,
}

server_config = {
    **client_config,
    #100
    "max_iterations": 100,
}

## Set up model etc.

In [9]:
device = server_config["device"]

model = ClassificationBNNLocalRepam(config=model_config)

# Initial parameters.
init_q_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model.num_parameters,)).to(device) 
    * client_config["init_var"] ** 0.5,
}

prior_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device),
    "scale": model_config["prior_var"] ** 0.5 
    * torch.ones(size=(model.num_parameters,)).to(device),
}

init_factor_nat_params = {
    "np1": torch.zeros(model.num_parameters).to(device),
    "np2": torch.zeros(model.num_parameters).to(device),
}

p = MeanFieldGaussianDistribution(
    std_params=prior_std_params, is_trainable=False
)
init_q = MeanFieldGaussianDistribution(
    std_params=init_q_std_params, is_trainable=False
)

clients = []
size_training = 0
for i in range(10):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params)
    clients.append(
        Client(
            data=data_i,
            model=model,
            t=t_i,
            config=client_config,
            val_data=test_data
        )
    )    
server = SequentialServer(model=model, p=p, clients=clients, config=server_config, init_q=init_q, data=train_data, val_data=test_data)

## Run PGVI with the Alpha R\'enyi divergence! 1 Hidden Layer

In [10]:
i = 0
while not server.should_stop():
    server.tick()

    # Obtain performance metrics.
    metrics = server.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Iterations: 32.
Time taken: 15723.141.
Test mll: -0.584. Test acc: 0.968.
Train mll: -0.220. Train acc: 0.984.



0it [00:00, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Alpha--Rényi divergence on exactly the same conditions, clients and data splits with $\eta=5e-4$

In [12]:
model_config2 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config2 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 5e-4},
    "sigma_optimiser_params": {"lr": 5e-4},
    "early_stopping": EarlyStopping(5, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 2500,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "AR",
    "alpha": 0.5,
}

server_config2 = {
    **client_config2,
    #100
    "max_iterations": 33,
}

In [13]:
device2 = server_config2["device"]

model2 = ClassificationBNNLocalRepam(config=model_config2)

# Initial parameters.
init_q_std_params2 = {
    "loc": torch.zeros(size=(model2.num_parameters,)).to(device2).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model2.num_parameters,)).to(device2) 
    * client_config2["init_var"] ** 0.5,
}

prior_std_params2 = {
    "loc": torch.zeros(size=(model2.num_parameters,)).to(device2),
    "scale": model_config2["prior_var"] ** 0.5 
    * torch.ones(size=(model2.num_parameters,)).to(device2),
}

init_factor_nat_params2 = {
    "np1": torch.zeros(model2.num_parameters).to(device2),
    "np2": torch.zeros(model2.num_parameters).to(device2),
}

p2 = MeanFieldGaussianDistribution(
    std_params=prior_std_params2, is_trainable=False
)
init_q2 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params2, is_trainable=False
)

clients2 = []
size_training = 0
for i in range(10):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params2)
    clients2.append(
        Client(
            data=data_i,
            model=model2,
            t=t_i,
            config=client_config2,
            val_data=test_data
        )
    )    
server2 = SequentialServer(model=model2, p=p2, clients=clients2, config=server_config2, init_q=init_q2, data=train_data, val_data=test_data)


In [14]:
i = 0
while not server2.should_stop():
    server2.tick()

    # Obtain performance metrics.
    metrics2 = server2.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics2["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics2["val_mll"], metrics2["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics2["train_mll"], metrics2["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Iterations: 32.
Time taken: 3413.965.
Test mll: -0.092. Test acc: 0.974.
Train mll: -0.080. Train acc: 0.981.



## $\eta=0.001$

In [16]:
model_config3 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config3 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr":1e-3},
    "sigma_optimiser_params": {"lr": 1e-3},
    "early_stopping": EarlyStopping(5, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 2500,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "AR",
    "alpha": 0.5,
}

server_config3 = {
    **client_config3,
    #100
    "max_iterations": 33,
}

In [17]:
device3 = server_config3["device"]

model3 = ClassificationBNNLocalRepam(config=model_config3)

# Initial parameters.
init_q_std_params3 = {
    "loc": torch.zeros(size=(model3.num_parameters,)).to(device3).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model3.num_parameters,)).to(device3) 
    * client_config3["init_var"] ** 0.5,
}

prior_std_params3 = {
    "loc": torch.zeros(size=(model3.num_parameters,)).to(device3),
    "scale": model_config3["prior_var"] ** 0.5 
    * torch.ones(size=(model3.num_parameters,)).to(device3),
}

init_factor_nat_params3 = {
    "np1": torch.zeros(model3.num_parameters).to(device3),
    "np2": torch.zeros(model3.num_parameters).to(device3),
}

p3 = MeanFieldGaussianDistribution(
    std_params=prior_std_params3, is_trainable=False
)
init_q3 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params3, is_trainable=False
)

clients3 = []
size_training = 0
for i in range(10):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params3)
    clients3.append(
        Client(
            data=data_i,
            model=model3,
            t=t_i,
            config=client_config3,
            val_data=test_data
        )
)    
server3 = SequentialServer(model=model3, p=p3, clients=clients3, config=server_config3, init_q=init_q3, data=train_data, val_data=test_data)


In [18]:
i = 0
while not server3.should_stop():
    server3.tick()

    # Obtain performance metrics.
    metrics3 = server3.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics3["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics3["val_mll"], metrics3["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics3["train_mll"], metrics3["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Iterations: 32.
Time taken: 14752.558.
Test mll: -0.524. Test acc: 0.964.
Train mll: -0.255. Train acc: 0.979.



## $\eta=2e-4$

In [19]:
model_config4 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config4 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr":2e-4},
    "sigma_optimiser_params": {"lr": 2e-4},
    "early_stopping": EarlyStopping(5, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 2500,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "AR",
    "alpha": 0.5,
}

server_config4 = {
    **client_config4,
    #100
    "max_iterations": 33,
}

In [20]:
device4 = server_config4["device"]

model4 = ClassificationBNNLocalRepam(config=model_config4)

# Initial parameters.
init_q_std_params4 = {
    "loc": torch.zeros(size=(model4.num_parameters,)).to(device4).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model4.num_parameters,)).to(device4) 
    * client_config4["init_var"] ** 0.5,
}

prior_std_params4 = {
    "loc": torch.zeros(size=(model4.num_parameters,)).to(device4),
    "scale": model_config4["prior_var"] ** 0.5 
    * torch.ones(size=(model4.num_parameters,)).to(device4),
}

init_factor_nat_params4 = {
    "np1": torch.zeros(model4.num_parameters).to(device4),
    "np2": torch.zeros(model4.num_parameters).to(device4),
}

p4 = MeanFieldGaussianDistribution(
    std_params=prior_std_params4, is_trainable=False
)
init_q4 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params4, is_trainable=False
)

clients4 = []
size_training = 0
for i in range(10):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params4)
    clients4.append(
        Client(
            data=data_i,
            model=model4,
            t=t_i,
            config=client_config4,
            val_data=test_data
        )
)    
server4 = SequentialServer(model=model4, p=p4, clients=clients4, config=server_config4, init_q=init_q4, data=train_data, val_data=test_data)


In [22]:
i = 0
while not server4.should_stop():
    server4.tick()

    # Obtain performance metrics.
    metrics4 = server4.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics4["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics4["val_mll"], metrics4["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics4["train_mll"], metrics4["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Iterations: 28.
Time taken: 28722.062.
Test mll: -0.098. Test acc: 0.973.
Train mll: -0.088. Train acc: 0.979.



## $\eta=0.005$

In [73]:
model_config5 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config5 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr":5e-3},
    "sigma_optimiser_params": {"lr": 5e-3},
    "early_stopping": EarlyStopping(5, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 2500,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
    "divergence": "AR",
    "alpha": 0.5,
}

server_config5 = {
    **client_config5,
    #100
    "max_iterations": 33,
}

In [75]:
device5 = server_config5["device"]

model5 = ClassificationBNNLocalRepam(config=model_config5)

# Initial parameters.
init_q_std_params5 = {
    "loc": torch.zeros(size=(model5.num_parameters,)).to(device5).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model5.num_parameters,)).to(device5) 
    * client_config5["init_var"] ** 0.5,
}

prior_std_params5 = {
    "loc": torch.zeros(size=(model5.num_parameters,)).to(device5),
    "scale": model_config5["prior_var"] ** 0.5 
    * torch.ones(size=(model5.num_parameters,)).to(device5),
}

init_factor_nat_params5 = {
    "np1": torch.zeros(model5.num_parameters).to(device5),
    "np2": torch.zeros(model5.num_parameters).to(device5),
}

p5 = MeanFieldGaussianDistribution(
    std_params=prior_std_params5, is_trainable=False
)
init_q5 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params5, is_trainable=False
)

clients5 = []
size_training = 0
for i in range(10):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params5)
    clients5.append(
        Client(
            data=data_i,
            model=model5,
            t=t_i,
            config=client_config5,
            val_data=test_data
        )
)    
server5 = SequentialServer(model=model5, p=p5, clients=clients5, config=server_config5, init_q=init_q5, data=train_data, val_data=test_data)


In [76]:
i = 0
while not server5.should_stop():
    server5.tick()

    # Obtain performance metrics.
    metrics5 = server5.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics5["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics5["val_mll"], metrics5["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics5["train_mll"], metrics5["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

Iterations: 32.
Time taken: 10022.458.
Test mll: -0.639. Test acc: 0.962.
Train mll: -0.256. Train acc: 0.982.



In [53]:
train_acc = []
val_acc = []
times = []
comms=[]
train_acc.append(100.0)
val_acc.append(100.0)
times.append(1.0)
comms.append(0)
for metrics in server.log["performance_metrics"]:
    train_acc.append(100-metrics["train_acc"]*100)
    val_acc.append(100-metrics["val_acc"]*100)
    times.append(metrics["time"])
    comms.append(metrics["communications"])

train_acc2 = []
val_acc2 = []
times2 = []
train_acc2.append(100.0)
val_acc2.append(100.0)
times2.append(1.0)
comms2=[]
comms2.append(0)
for metrics in server2.log["performance_metrics"]:
    train_acc2.append(100-metrics["train_acc"]*100)
    val_acc2.append(100-metrics["val_acc"]*100)
    times2.append(metrics["time"])
    comms2.append(metrics["communications"])
train_acc3 = []
val_acc3 = []
times3 = []
comms3 = []
train_acc3.append(100.0)
val_acc3.append(100.0)
times3.append(1.0)
comms3.append(0)
for metrics in server3.log["performance_metrics"]:
    train_acc3.append(100-metrics["train_acc"]*100)
    val_acc3.append(100-metrics["val_acc"]*100)
    times3.append(metrics["time"])
    comms3.append(metrics["communications"])
    
train_acc4 = []
val_acc4 = []
times4 = []
comms4=[]
train_acc4.append(100.0)
val_acc4.append(100.0)
times4.append(1.0)
comms4.append(0)
for metrics in server4.log["performance_metrics"]:
    train_acc4.append(100-metrics["train_acc"]*100)
    val_acc4.append(100-metrics["val_acc"]*100)
    times4.append(metrics["time"])
    comms4.append(metrics["communications"])
plt.figure(figsize=(6,4))
#plt.xscale("log")
plt.ylim(0,10)
plt.plot(comms, val_acc, linewidth=1., label=r"learning rate $\eta=2e-3$")
plt.plot(comms3, val_acc3, linewidth=1., label=r"learning rate $\eta=1e-3$")
plt.plot(comms2, val_acc2, linewidth=1., label=r"learning rate $\eta=5e-4$")
plt.plot(comms4, val_acc4, linewidth=1., label=r"learning rate $\eta=2e-4$")
plt.xlabel("Communication Rounds")
plt.ylabel("% Testing Error")
plt.legend(loc="upper right", frameon=False)

plt.grid(visible=True)
#plt.savefig('PGVI_on_MNIST_lrs.pgf', pad_inches=0)
plt.show()

/var/folders/6x/7wzwn4g54n38_8rfbfx611n00000gn/T/ipykernel_52910/3115237292.py:68: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [71]:
print("Validation Accuracy:")
print(f"lr=2e-3:", 100-np.min(val_acc))
print(f"lr=1e-3:", 100-np.min(val_acc3))
print(f"lr=5e-4:", 100-np.min(val_acc2))
print(f"lr=2e-4:", 100-np.min(val_acc4))

Validation Accuracy:
lr=2e-3: 97.44999694824219
lr=1e-3: 97.3499984741211
lr=5e-4: 97.47000122070312
lr=2e-4: 97.3499984741211


In [72]:
print("Training Accuracy:")
print(f"lr=2e-3:", 100-np.min(train_acc))
print(f"lr=1e-3:", 100-np.min(train_acc3))
print(f"lr=5e-4:", 100-np.min(train_acc2))
print(f"lr=2e-4:", 100-np.min(train_acc4))

Training Accuracy:
lr=2e-3: 98.5050048828125
lr=1e-3: 98.0
lr=5e-4: 98.125
lr=2e-4: 97.90833282470703
